In [1]:
import json
import torch
import numpy as np
from sklearn.model_selection import train_test_split
from datasets import Dataset, DatasetDict
from transformers import AutoTokenizer

data_path = r'C:\Users\Subways-Sun\OneDrive\Documents\GitHub\sodium-ion-batteries\data_annotated\search_20241106-223705_sodium+ion+battery+anode-sodium+ion+battery+cathode-sodium+ion+battery+electrode_annotated_rephrased.json'
with open(data_path, 'r', encoding='utf-8') as f:
    data = json.load(f)

texts = []
labels = []

for i in data:
    texts.append(i["text"])
    labels.append(i["label_int"])

train_texts, val_texts, train_labels, val_labels = train_test_split(
    texts,
    labels,
    test_size=0.2,
    random_state=42
)

train_dataset = Dataset.from_dict({
    'text': train_texts,
    'labels': train_labels
})

val_dataset = Dataset.from_dict({
    'text': val_texts,
    'labels': val_labels
})

raw_datasets = DatasetDict({
    'train': train_dataset,
    'validation': val_dataset
})
print(len(texts))

400


In [2]:
print(raw_datasets)

DatasetDict({
    train: Dataset({
        features: ['text', 'labels'],
        num_rows: 320
    })
    validation: Dataset({
        features: ['text', 'labels'],
        num_rows: 80
    })
})


In [3]:
model_name = "batterydata/batterybert-cased-abstract"
tokenizer = AutoTokenizer.from_pretrained(model_name)

def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)

Map:   0%|          | 0/320 [00:00<?, ? examples/s]

Map:   0%|          | 0/80 [00:00<?, ? examples/s]

In [4]:
from transformers import AutoModelForSequenceClassification
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2, torch_dtype="auto")

In [5]:
from transformers import TrainingArguments

training_args = TrainingArguments(output_dir="test_trainer")

In [6]:
import evaluate

metric = evaluate.load("accuracy")

In [7]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [8]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(output_dir="test_trainer",
                                  eval_strategy="epoch",
                                  num_train_epochs=7)

In [9]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    compute_metrics=compute_metrics,
)


In [10]:
trainer.train()

  0%|          | 0/280 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

{'eval_loss': 0.49558788537979126, 'eval_accuracy': 0.825, 'eval_runtime': 0.6589, 'eval_samples_per_second': 121.415, 'eval_steps_per_second': 15.177, 'epoch': 1.0}


  0%|          | 0/10 [00:00<?, ?it/s]

{'eval_loss': 0.48762744665145874, 'eval_accuracy': 0.825, 'eval_runtime': 0.6456, 'eval_samples_per_second': 123.909, 'eval_steps_per_second': 15.489, 'epoch': 2.0}


...

  0%|          | 0/10 [00:00<?, ?it/s]

{'eval_loss': 0.2970426082611084, 'eval_accuracy': 0.9375, 'eval_runtime': 0.651, 'eval_samples_per_second': 122.888, 'eval_steps_per_second': 15.361, 'epoch': 6.0}


  0%|          | 0/10 [00:00<?, ?it/s]

{'eval_loss': 0.26735493540763855, 'eval_accuracy': 0.9375, 'eval_runtime': 0.6203, 'eval_samples_per_second': 128.962, 'eval_steps_per_second': 16.12, 'epoch': 7.0}
{'train_runtime': 62.7974, 'train_samples_per_second': 35.67, 'train_steps_per_second': 4.459, 'train_loss': 0.3904061453683036, 'epoch': 7.0}


TrainOutput(global_step=280, training_loss=0.3904061453683036, metrics={'train_runtime': 62.7974, 'train_samples_per_second': 35.67, 'train_steps_per_second': 4.459, 'total_flos': 589368764006400.0, 'train_loss': 0.3904061453683036, 'epoch': 7.0})

In [11]:
trainer.save_model("batterybert-cased-abstract_finetuned_400")